In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline
from pylab import rcParams

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.cross_validation import cross_val_score as cvs
from sklearn.model_selection import cross_val_predict as cross_val_predict
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.metrics import log_loss
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.ensemble import IsolationForest

rcParams['figure.figsize'] = 15, 10
%config InlineBackend.figure_format = 'retina'  # более качественное отображение графиков

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # вывод всех запрошенных переменных ячейки

Загрузка данных

In [ ]:
DF = pd.read_csv("train.csv", index_col="Time")
DF.head()

In [ ]:
list_of_test = []
for i in range(0, 10):
    test = pd.read_csv('./test/0%d_test.csv'%i, index_col="Time")
    #print(len(test), end=';  ')
    test.dropna(inplace=True)
    #print(len(test))
    list_of_test.append(test)
for i in range(10, 48):
    test = pd.read_csv('./test/%d_test.csv'%i, index_col="Time")
    #print(len(test), end=';  ')
    test.dropna(inplace=True)
    #print(len(test))
    list_of_test.append(test)
    
test_all = list_of_test[0]
for i in range(1, len(list_of_test)):
    test_all = test_all.append(list_of_test[i])

Объединяем всё и масштабируем

In [ ]:
scaler = StandardScaler()
table = DF.append(test_all)
scaler.fit(table)

table = scaler.transform(table)
DF = scaler.transform(DF)
test_all = scaler.transform(test_all)

Пихаем в kmeans

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=141).fit(table)
predict = kmeans.predict(table)

Сохраняем ответ в файлах

In [ ]:
np.save("train_clusters", predict[:len(DF)])
limit = len(DF)
for i in range(len(list_of_test)):
    np.save("test" + str(i) + "_clusters", predict[limit:limit+len(list_of_test[i])])
    limit += len(list_of_test[i])

На этом решение почти окончено. Грузим ответы из файла и ищем промежутки из 100 ответов kmeans, которые не встречаются на трейне. Если такое встретилось 100 раз за 10000 единиц времени - бьём в колокола.

In [ ]:
W = 100
train = np.load("clusters//train_clusters.npy")
p = np.chararray.tostring(train)

In [ ]:
ans = pd.DataFrame(index=range(48))
ans[0] = 0

for test_i in range(48):
    original_test = pd.read_csv('./test/' + str(test_i // 10) + str(test_i % 10) + '_test.csv', index_col="Time").dropna()
    
    test = np.load("clusters//test" + str(test_i) + "_clusters.npy")
    t = np.chararray.tostring(test)
    
    candidates = []
    for i in range(W, len(test)):
        j = p.find(t[(i - W) * 4:i * 4])
        if j == -1:
            candidates.append(original_test.index[i])
            
            if len(candidates) >= W and candidates[-1] - candidates[-W] < 10000:
                ans[0][test_i] = original_test.index[i]
                print(test_i, ") ", i, ans[0][test_i])
                break
                
    if ans[0][test_i] == 0:
        print("anomal!")
        ans[0][test_i] = candidates[0]

In [ ]:
ans.to_csv("answer.csv", header=None)